In [ ]:
import sys
from pathlib import Path

import synthcity.logger as log
from synthcity.benchmark import Benchmarks
from synthcity.plugins import Plugins
from synthcity.utils.serialization import load_from_file, save_to_file
from synthcity.plugins.core.dataloader import SurvivalAnalysisDataLoader

from datasets import get_dataset

log.add(sink=sys.stderr, level="DEBUG")

plugins = Plugins().list()

out_dir = Path("output")

gain_scenarios = [
    (
        "uncensoring_with_date",
        {
            "uncensoring_model": "date",
            "tte_strategy": "uncensoring",
            "dataloader_sampling_strategy": "none",
            "use_survival_conditional": False,
        },
    ),
    (
        "survival_function_with_date",
        {
            "uncensoring_model": "date",
            "tte_strategy": "survival_function",
            "dataloader_sampling_strategy": "none",
            "use_survival_conditional": False,
        },
    ),
    (
        "survival_function_regression_no_sampling",
        {
            "uncensoring_model": "survival_function_regression",
            "tte_strategy": "survival_function",
            "dataloader_sampling_strategy": "none",
            "use_survival_conditional": False,
        },
    ),
    (
        "survival_function_regression_imbalanced_cens_sampling",
        {
            "uncensoring_model": "survival_function_regression",
            "tte_strategy": "survival_function",
            "dataloader_sampling_strategy": "imbalanced_censoring",
            "use_survival_conditional": False,
        },
    ),
    (
        "survival_function_regression_imbalanced_time_cens_no_cond",
        {
            "uncensoring_model": "survival_function_regression",
            "tte_strategy": "survival_function",
            "dataloader_sampling_strategy": "imbalanced_time_censoring",
            "use_survival_conditional": False,
        },
    ),
    (
        "survival_function_regression_imbalanced_time_cens_sampling",
        {
            "uncensoring_model": "survival_function_regression",
            "tte_strategy": "survival_function",
            "dataloader_sampling_strategy": "imbalanced_time_censoring",
        },
    ),
]


def evaluate_dataset(dataset: str, scenarios: list):
    df, duration_col, event_col, time_horizons = get_dataset(dataset)
    repeats = 1
    
    # experiment = "gain_of_function_parametric"
    experiment = "sources_of_gain_parametric"
    for scenario_name, scenario_args in scenarios:
        bkp = out_dir / f"experiment_{experiment}_{dataset}_{scenario_name}_{repeats}.bkp"

        score = Benchmarks.evaluate(
            [(scenario_name, "survival_gan", scenario_args)],
            SurvivalAnalysisDataLoader(
                df,
                target_column=event_col,
                time_to_event_column=duration_col,
                time_horizons=time_horizons,
            ),
            task_type="survival_analysis",
            synthetic_size=len(df),
            repeats=repeats,
        )
        save_to_file(bkp, score)

        print("Scenario", scenario_name, scenario_args)
        Benchmarks.print(score)

## AIDS dataset

In [ ]:
base_score = evaluate_dataset("aids", gain_scenarios)